# Snowpark Lazy Evaluation

Snowpark SQL statements are executed lazily on the server, which reduces the amount of data transferred between your client and Snowflake.

The core abstraction in Snowpark is the DataFrame, which represents a set of data and provides methods to operate on that data. In your client code, you construct a DataFrame object and specify the data that you want to make use of (e.g., the columns containing the data, the filter to apply to rows, aggregations, etc).

The data is not retrieved at the time when you construct the DataFrame object. Instead, when you are ready to retrieve the data, you can perform an action that evaluates the DataFrame objects and sends the corresponding SQL statements to Snowflake for execution.

---

## Create a DataFrame from a SQL Query

To construct a DataFrame from a SQL query,  use the `sql` method in the Session class. 


### Create a Session
---

Create a Snowpark Session by passing in the connection properties file created in the [first lab exercise](../A-Dataframes/01-Sessions.ipynb).

In [ ]:
import com.snowflake.snowpark._
import com.snowflake.snowpark.functions._
import com.snowflake.snowpark.types._

// Set connection properties file variable
val pwd = sys.env.get("PWD").fold("")(_.toString)
val filename = s"$pwd/de_snowpark/connect.properties"

val session = Session.builder.configFile(s"$filename").create


Create a DataFrame that details the warehouses in your account for which you have access privileges.

In [ ]:
val warehouseDF = session.sql("show warehouses")

### Retrieving Column Definitions

To retrieve the definition of the columns in the dataset for the DataFrame, call the schema method. This method returns a StructType object that contains an Array of StructField objects. Each StructField object contains the definition of a column.

Call the schema function on the warehouseDF DataFrame to see the column names of the `show` command output as a table.


In [ ]:
warehouseDF.schema

Actions cause the DataFrame to be evaluated. When you call a method that performs an action, Snowpark sends the SQL query for the DataFrame to the server for evaluation.  

---

### Evaluate the Dataframe 

Run the `count` and `show` actions to cause the `warehouseDF` DataFrame to be evaluated.

Examine the ```SnowflakePlan``` and view the SQL query Snowpark sends to Snowflake for evaluation.

Note the addition of escape characters in the following to wrap column identifiers which are not upper case in Snowflake.


In [ ]:
val num = warehouseDF.count()
println ( s"Number of warehouses: $num")

warehouseDF.select( col("\"name\""),col("\"state\""),col("\"size\"")).show()

### Perform Operations Using a DataFrame

Filter the `warehouseDF` DataFrame and execute the following SQL command to suspend any warehouse in the STARTED state.

In [ ]:
val startedWarehouseDF = warehouseDF.select( col("\"name\""),col("\"state\""),col("\"size\""))
    .filter(col("\"state\"") === "STARTED" )

var startedWarehouseArray = startedWarehouseDF.collect()


for (wh <- startedWarehouseArray) {
    var name = wh.get(0)
    println(s"Suspending warehouse: $name ")
    session.sql(s"alter warehouse $name suspend").collect
}


---
## Create a DataFrame from a Table

To construct a DataFrame on a table,  use the `table` method in the Session class. 

Note that this same method is also used for Views and Streams.

In [ ]:
val onTimeFlightsDF = session.table("raw.ONTIME_REPORTING")

Evaluate the DataFrame by calling the `count` method:

In [ ]:
onTimeFlightsDF.count()

---
## Create a DataFrame from a Sequence

To construct a DataFrame from a sequence use the `createDataFrame` method in the Session class. 

In [ ]:
val carrierLookupDF = session.createDataFrame( Seq(("AA","American Airlines Inc."), 
                                         ("FDN","SnowBear Air "),
                                         ("G4","Allegiant Air"),
                                         ("EV","ExpressJet Airlines LLC"),
                                         ("UA","United Air Lines Inc."),
                                         ("NK","Spirit Air Lines"),
                                         ("OO","SkyWest Airlines Inc."),
                                         ("HA","Hawaiian Airlines Inc."),
                                         ("OH","PSA Airlines Inc."),
                                         ("YX","Republic Airline"),
                                         ("AS","Alaska Airlines Inc."),
                                         ("9E","Endeavor Air Inc."), 
                                        )).toDF("CODE", "DESCRIPTION")



Evaluate the DataFrame by calling the `show` method.

In [ ]:
carrierLookupDF.show()